<center><h1><b>ĐỒ ÁN: EEDI - MINING MISCONCEPTIONS IN MATHEMATICS</b></h1><center>

<center><h2><u>Môn:</u> Học sâu cho Khoa học dữ liệu - <u>Lớp:</u> 21_21</h2></center>

<h3><u>Giáo viên hướng dẫn:</u><h3>

- TS. Nguyễn Tiến Huy
- TS. Lê Thanh Tùng
- ThS. Nguyễn Trần Duy Minh  

<h3><u>Nhóm thực hiện:</u> Nhóm 1</h3>

| Họ và tên | MSSV |
| :--- | :---:|
| Phạm Nhật Duy | 21120058 |
| Nguyễn Trúc Nguyên | 21120102 |
| Trương Công Trung | 21120158 |
| Lê Trần Minh Khuê | 21120279 |

# **Tổng quan**

<u>Trong phạm vi đồ án này</u>, nhóm chúng em sẽ phát triển một mô hình học máy cung cấp khả năng phân tích và dự đoán mối liên hệ giữa *các ngộ nhận* với *các câu trả lời không chính xác* trong các câu hỏi trắc nghiệm.

<u>Mục tiêu:</u> Đối với mỗi câu hỏi trắc nghiệm, mô hình sẽ

- Xác định được câu trả lời đúng,
- Xác định được những lầm tưởng/ ngộ nhận có thể dẫn đến việc chọn đáp án sai.

<u>Trong phạm vi file này</u>, nhóm sẽ tiến hành pretrain mô hình ngôn ngữ lớn (Qwen2.5-14B) với bộ dữ liệu Eedi.

In [ ]:
%%time
! python -m pip install -q transformers peft bitsandbytes accelerate datasets trl optimum triton

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 7.7 MB/s eta 0:00:00:00:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.1 MB/s eta 0:00:00
CPU times: user 223 ms, sys: 55.3 ms, total: 279 ms
Wall time: 14.2 s


In [ ]:
%%time
!cp -r /kaggle/input/eedi-dependencies/AutoAWQ ./AutoAWQ
!pip install ./AutoAWQ -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 42.6 MB/s eta 0:00:0000:0100:01
CPU times: user 159 ms, sys: 48.6 ms, total: 208 ms
Wall time: 12.6 s


In [ ]:
! pip list | grep -e transformers -e cuda -e vllm -e autoawq -e torch -e grpcio -e trl -e triton -e optimum

autoawq                            0.2.7.post2
cuda-python                        12.2.1
cupy-cuda12x                       12.2.0
dask-cuda                          24.12.0
fastrlock                          0.8.2
grpcio                             1.68.1
grpcio-status                      1.48.2
jax-cuda12-pjrt                    0.4.33
jax-cuda12-plugin                  0.4.33
numba-cuda                         0.0.17.1
nvidia-cuda-cupti-cu12             12.6.80
nvidia-cuda-nvcc-cu12              12.6.85
nvidia-cuda-runtime-cu12           12.6.77
optimum                            1.23.3
pycuda                             2024.1.2
pytorch-ignite                     0.5.1
pytorch-lightning                  2.5.0.post0
sentence-transformers              3.3.1
torch                              2.5.1+cu121
torchaudio                         2.5.1+cu121
torchinfo                          1.8.0
torchmetrics                       1.6.1
torchsummary                       1.5.1
torchtune   

# Chuẩn bị dữ liệu

In [ ]:
import pandas as pd

df_train = pd.read_csv("/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv").fillna(-1)

In [ ]:
items = []
rows = []
for _, row in df_train.iterrows():
    for choice in ['A', 'B', 'C', 'D']:
        if choice == row["CorrectAnswer"]:
            continue

        correct_answer = row[f"Answer{row['CorrectAnswer']}Text"]

        query_text = (f"Subject name: {row['SubjectName']}\n"
                      f"Construct name: {row['ConstructName']}\n"
                      f"Question: {row['QuestionText']}\n"
                      f"Correct answer: {correct_answer}\n"
                      f"### Misconcepte incorrect answer: {choice}.{row[f'Answer{choice}Text']}")

        rows.append({
            "query_text": query_text,
            "QuestionId_Answer": f"{row['QuestionId']}_{choice}",
            "ConstructName": row["ConstructName"],
            "SubjectName": row["SubjectName"],
            "QuestionText": row["QuestionText"],
            "correct_answer": correct_answer,
            "incorrect_answer": row[f"Answer{choice}Text"]
        })

df = pd.DataFrame(rows)

In [ ]:
from datasets import load_dataset, Dataset

def create_messages(row):
    messages = [
        {
            "content": row["query_text"],
            "role": "user"
        },
        {
            "content": "Yes",
            "role": "assistant"
        }
    ]
    return messages

df["messages"] = df.apply(create_messages, axis=1)
dataset = Dataset.from_pandas(df[["messages"]])
dataset[0]

{'messages': [{'content': 'Subject name: BIDMAS\nConstruct name: Use the order of operations to carry out calculations involving powers\nQuestion: \\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?\nCorrect answer: \\( 3 \\times(2+4)-5 \\)\n### Misconcepte incorrect answer: B.\\( 3 \\times 2+(4-5) \\)',
   'role': 'user'},
  {'content': 'Yes', 'role': 'assistant'}]}

In [ ]:
len(dataset)

5607

# Load mô hình

+ Nhóm sử dụng một mô hình Qwen phiên bản 2.5 với 14 tỷ tham số, thuộc loại "Instruct", được huấn luyện đặc biệt để thực hiện theo hướng dẫn.

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"

from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, PeftModel


model_name = "Qwen/Qwen2.5-14B-Instruct-AWQ"
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/107k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.02G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

+ Thiết lập môi trường để theo dõi quá trình huấn luyện mô hình bằng Weights & Biases.

In [ ]:
from kaggle_secrets import UserSecretsClient
# from google.colab import userdata
import wandb

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_api")
wandb.login(key=secret_value_0)
wandb.init(project='eedi', job_type='train', name='qwen14b')

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: phamduy1032003 (phamduy1032003-tr-ng-i-h-c-khoa-h-c-t-nhi-n-hqg-hcm). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-14B-Instruct-AWQ', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|object_ref_start|>', '<|object_ref_end|>', '<|box_start|>', '<|box_end|>', '<|quad_start|>', '<|quad_end|>', '<|vision_start|>', '<|vision_end|>', '<|vision_pad|>', '<|image_pad|>', '<|video_pad|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=Fal

+ Dữ liệu được định dạng thành dạng hội thoại với các thẻ đặc biệt để mô hình hiểu vai trò của các thành phần.

In [ ]:
tokenizer.apply_chat_template(dataset[0]['messages'], tokenize=False)

'<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nSubject name: BIDMAS\nConstruct name: Use the order of operations to carry out calculations involving powers\nQuestion: \\[\n3 \\times 2+4-5\n\\]\nWhere do the brackets need to go to make the answer equal \\( 13 \\) ?\nCorrect answer: \\( 3 \\times(2+4)-5 \\)\n### Misconcepte incorrect answer: B.\\( 3 \\times 2+(4-5) \\)<|im_end|>\n<|im_start|>assistant\nYes<|im_end|>\n'

In [ ]:
def formatting_prompts_func(example):
    return tokenizer.apply_chat_template(example['messages'], tokenize=False)

response_template = "<|im_start|>assistant\n"
collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
    padding_free=True,
)

+ Khởi tạo một SFTTrainer (Supervised Fine-Tuning Trainer) để huấn luyện mô hình ngôn ngữ lớn (LLM) với cấu hình tinh chỉnh LoRA (Low-Rank Adaptation).

# Huấn luyện mô hình

In [ ]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",
    inference_mode=False,
    r=4,
    lora_alpha=8,
    lora_dropout=0.05,
    target_modules=["q_proj", "k_proj"],
    bias="none"
)


training_args = SFTConfig(
    packing=False,
    max_seq_length=512,
    output_dir="/tmp",
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.001,
    eval_strategy="steps",
    eval_steps=5,
    save_strategy="steps",
    save_steps=999999,
    gradient_accumulation_steps=16,
    remove_unused_columns=True,
    fp16=True,
    logging_strategy="steps",
    logging_steps=1,
    lr_scheduler_type='cosine',
    # warmup_ratio=0.03,
    report_to='none'
)

trainer = SFTTrainer(
    base_model,
    train_dataset=dataset.select(range(680)),
    eval_dataset=dataset.select(range(34)),
    args=training_args,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    data_collator=collator
)

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Map:   0%|          | 0/34 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
5,254.466000,15.451650
10,235.776600,13.822470
15,179.999800,10.149148
20,108.127000,5.990427
25,64.483400,3.737451
30,42.499100,2.599216
35,32.441900,1.775441
40,22.887700,1.351074


TrainOutput(global_step=42, training_loss=124.33512278965541, metrics={'train_runtime': 2053.6936, 'train_samples_per_second': 0.331, 'train_steps_per_second': 0.02, 'total_flos': 436374648317952.0, 'train_loss': 124.33512278965541, 'epoch': 0.9882352941176471})

# Lưu mô hình

+ Lưu checkpoint cuối cùng của mô hình và đồng bộ hóa kết quả với nền tảng Weights & Biases để sử dụng huấn luyện tiếp sau này.

In [ ]:
print("Saving last checkpoint of the model")
output_name = "./qwen2-14b"
# model.save_pretrained(output_name + "/last_checkpoint")
trainer.save_model(output_name + "/last_checkpoint")
tokenizer.save_pretrained(output_name + "/last_checkpoint")

artifact = wandb.Artifact('qwen2-14b', type='model')

artifact.add_dir(output_name, 'qwen2-14b')
wandb.log_artifact(artifact)

wandb.finish()

Saving last checkpoint of the model


wandb: Adding directory to artifact (./qwen2-14b)... Done. 0.1s
